In [131]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import shutil
import random
import sys
import time
import re
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize


In [2]:
lyrics = pd.read_csv("song_lyrics.csv", sep = ",")

# EDA

In [3]:
lyrics.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [4]:
lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5134856 entries, 0 to 5134855
Data columns (total 11 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   title          object
 1   tag            object
 2   artist         object
 3   year           int64 
 4   views          int64 
 5   features       object
 6   lyrics         object
 7   id             int64 
 8   language_cld3  object
 9   language_ft    object
 10  language       object
dtypes: int64(3), object(8)
memory usage: 430.9+ MB


In [55]:
#see max and min year
print(min(df["year"]))
print(max(df["year"]))

1
2023


# Data filtering

In [106]:
#filter on a language English
lyrics_en = lyrics.loc[lyrics['language'] == "en"]

#only use songs from 1960 to 2023
lyrics_en = lyrics_en.loc[lyrics_en['year'] > 1959]
lyrics_en = lyrics_en.loc[lyrics_en['year'] < 2023]

In [128]:
lyrics_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3338151 entries, 0 to 5134855
Data columns (total 11 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   title          object
 1   tag            object
 2   artist         object
 3   year           int64 
 4   views          int64 
 5   features       object
 6   lyrics         object
 7   id             int64 
 8   language_cld3  object
 9   language_ft    object
 10  language       object
dtypes: int64(3), object(8)
memory usage: 305.6+ MB


In [129]:
lyrics_en = lyrics_en.loc[lyrics_en['tag'] != "misc"]

In [130]:
lyrics_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3217307 entries, 0 to 5134855
Data columns (total 11 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   title          object
 1   tag            object
 2   artist         object
 3   year           int64 
 4   views          int64 
 5   features       object
 6   lyrics         object
 7   id             int64 
 8   language_cld3  object
 9   language_ft    object
 10  language       object
dtypes: int64(3), object(8)
memory usage: 294.6+ MB


In [ ]:
lyrics_en = lyrics_en.loc[lyrics_en['tag'] != "misc"]

In [132]:
# Tokenize the lyrics and count the number of words
lyrics_length = lyrics_en['lyrics'].apply(lambda x: len(word_tokenize(x))) 

average_length = lyrics_length.mean()
print(f"Average lyrics length: {average_length:.2f} words")

Average lyrics length: 339.78 words


In [136]:
print(max(lyrics_length))
print(min(lyrics_length))

20908
3


In [137]:
# Filter rows based on the lyrics length
lyrics_en = lyrics_en.loc[lyrics_length > 10]
lyrics_en = lyrics_en.loc[lyrics_length < 1560]


In [138]:
lyrics_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3212508 entries, 0 to 5134855
Data columns (total 11 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   title          object
 1   tag            object
 2   artist         object
 3   year           int64 
 4   views          int64 
 5   features       object
 6   lyrics         object
 7   id             int64 
 8   language_cld3  object
 9   language_ft    object
 10  language       object
dtypes: int64(3), object(8)
memory usage: 294.1+ MB


In [148]:
#filter the yearly top 1000 songs based on the views
top_songs = lyrics_en.sort_values(['year', 'views'], ascending=[True, False])\
              .groupby('year').head(1000)

In [149]:
top_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63000 entries, 66816 to 5041505
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          62999 non-null  object
 1   tag            63000 non-null  object
 2   artist         63000 non-null  object
 3   year           63000 non-null  int64 
 4   views          63000 non-null  int64 
 5   features       63000 non-null  object
 6   lyrics         63000 non-null  object
 7   id             63000 non-null  int64 
 8   language_cld3  63000 non-null  object
 9   language_ft    63000 non-null  object
 10  language       63000 non-null  object
dtypes: int64(3), object(8)
memory usage: 5.8+ MB


In [150]:
#check duplicates
num_duplicates = top_songs.duplicated(subset=['title', 'artist']).sum()
print(num_duplicates)

0


In [155]:
#drop rows with NaN values
top_songs = top_songs.dropna()

In [156]:
top_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62999 entries, 66816 to 5041505
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          62999 non-null  object
 1   tag            62999 non-null  object
 2   artist         62999 non-null  object
 3   year           62999 non-null  int64 
 4   views          62999 non-null  int64 
 5   features       62999 non-null  object
 6   lyrics         62999 non-null  object
 7   id             62999 non-null  int64 
 8   language_cld3  62999 non-null  object
 9   language_ft    62999 non-null  object
 10  language       62999 non-null  object
dtypes: int64(3), object(8)
memory usage: 5.8+ MB


In [157]:
#remove extra columns
drop_columns = ['features', 'id', 'language_cld3', 'language_ft']  # List of column names to be dropped
top_songs.drop(drop_columns, axis=1, inplace=True)

In [158]:
top_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62999 entries, 66816 to 5041505
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     62999 non-null  object
 1   tag       62999 non-null  object
 2   artist    62999 non-null  object
 3   year      62999 non-null  int64 
 4   views     62999 non-null  int64 
 5   lyrics    62999 non-null  object
 6   language  62999 non-null  object
dtypes: int64(2), object(5)
memory usage: 3.8+ MB


In [159]:
# check unique values
count_number_songs = top_songs.groupby(['title', 'artist']).ngroups
print("Total count of unique songs:", count_value_total)

count_number_artists = top_songs.groupby(['artist']).ngroups
print("Total count of unique artists:", count_number_artists)

Total count of unique songs: 62999
Total count of unique artists: 11125


In [160]:
#see max and min views
print(min(top_songs["views"]))
print(max(top_songs["views"]))

204
16003444


In [162]:
#lyrics before cleaning
print(top_songs["lyrics"][0])

# the song sections are marked by e.g. [Chorus: Opera Steve & Cam'ron] & [Verse 3] which should be removed

[Chorus: Opera Steve & Cam'ron]
Killa Cam, Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam
Killa Cam, Cam, Killa (Killa!)
Killa Cam, Killa Cam, Cam (Bases loaded)
Killa Cam, Killa Cam (Uh-huh)
Killa Cam, Cam (Santana on second, Jim on third)
Killa Cam, Killa Cam, Cam (I'm at bat)
Killa Killa Killa Cam
Killa Cam, Cam, Killa (I'm 'bout to hit this shit out the world)
Killa Cam (Ugh, Heatmakerz), Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam (Hahahaha)
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam
Killa Cam, Cam, Killa (We  make this shit clap)
Killa Cam, Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam (Killa! Killa!)
Killa Cam, Cam, Killa
[Verse 1]
With the goons I spar, stay in tune with ma (What up?)
She like, "Damn, this the realest since 'Kumbaya'"
Bomaye, Killa Cam, my Lord (My Lord)
Still the man with the pan, scrilla, fam, on board
Now bitches, they want to neuter me, nigga

In [163]:
# Clean square brackets from lyrics
def clean_lyrics(lyrics):
    cleaned_lyrics = re.sub(r'\[.*?\]', '', lyrics)  # Removes square brackets and their contents
    cleaned_lyrics = cleaned_lyrics.strip()  # Removes leading and trailing whitespace
    return cleaned_lyrics

top_songs["lyrics"] = top_songs["lyrics"].apply(clean_lyrics)


In [164]:
#lyrics after cleaning
print(top_songs["lyrics"][0])

Killa Cam, Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam
Killa Cam, Cam, Killa (Killa!)
Killa Cam, Killa Cam, Cam (Bases loaded)
Killa Cam, Killa Cam (Uh-huh)
Killa Cam, Cam (Santana on second, Jim on third)
Killa Cam, Killa Cam, Cam (I'm at bat)
Killa Killa Killa Cam
Killa Cam, Cam, Killa (I'm 'bout to hit this shit out the world)
Killa Cam (Ugh, Heatmakerz), Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam (Hahahaha)
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam
Killa Cam, Cam, Killa (We  make this shit clap)
Killa Cam, Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam (Killa! Killa!)
Killa Cam, Cam, Killa

With the goons I spar, stay in tune with ma (What up?)
She like, "Damn, this the realest since 'Kumbaya'"
Bomaye, Killa Cam, my Lord (My Lord)
Still the man with the pan, scrilla, fam, on board
Now bitches, they want to neuter me, niggas, they want to tutor me
The hooligan in 

In [165]:
top_songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62999 entries, 66816 to 5041505
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     62999 non-null  object
 1   tag       62999 non-null  object
 2   artist    62999 non-null  object
 3   year      62999 non-null  int64 
 4   views     62999 non-null  int64 
 5   lyrics    62999 non-null  object
 6   language  62999 non-null  object
dtypes: int64(2), object(5)
memory usage: 5.9+ MB


In [166]:
#export the dataframe into csv files
top_songs.to_csv('lyrics_top_songs_dataset.csv', index=False)